In [1]:
from bs4 import BeautifulSoup
import feedparser, urllib, json, os, re, requests
from collections import defaultdict

from selenium import webdriver


In [85]:
#need to install a drive to use for selenium (e.g. chromediver)
DRIVER_PATH='../chromedriver'

driver = webdriver.Chrome(DRIVER_PATH)
driver.get('https://portal.higg.org/')

#clicking the login button to bring us to the creds page
driver.find_element_by_xpath("/html/body/app-root/mat-drawer-container/mat-drawer-content/section/div/main/ng-component/div/div[2]/div/div/app-sign-in/div/div/div[2]/div/button").click()

In [88]:
email_input = '/html/body/div/div/div/form/div/div/div/div/div[2]/div[2]/span/div/div/div/div/div/div/div/div/div/div/div[1]/div/input'
pass_input = '/html/body/div/div/div/form/div/div/div/div/div[2]/div[2]/span/div/div/div/div/div/div/div/div/div/div/div[2]/div/div/input'

#find the login fields for email and password 
log_email = driver.find_element_by_xpath(email_input)
log_pass = driver.find_element_by_xpath(pass_input)

#send your personal login info to the driver 
log_email.send_keys("")
log_pass.send_keys("")

In [90]:
#new path for the login button
login_path2 = '/html/body/div/div/div/form/div/div/div/button'

#second login click
driver.find_element_by_xpath(login_path2).click()


#expand dropdown list on the landing page
driver.find_element_by_xpath('/html/body/app-root/mat-drawer-container/mat-drawer-content/app-header/div/header/app-top-navbar/nav/div[1]/ul/li[4]/button').click()

#link to the MSI lib page
driver.find_element_by_partial_link_text('MSI').click()

#click on example materials page
driver.find_element_by_xpath('/html/body/app-root/mat-drawer-container/mat-drawer-content/app-header/div/msi-navbar/div/span[3]').click()


In [122]:
#k-link k-pager-nav
data = {'MAT_FAMILY':[], 'MATERIAL':[], 'Global Warming':[],
 'Eutrophication':[],
 'Water Scarcity':[],
 'Resource Depletion, Fossil Fuels':[]}

#each material is stored in a table on the page, with only 10 or so displayed, so we need to cycle through the table tabs
for match_i in driver.find_elements_by_class_name('k-link'):
    
    #check that it is a next-page link
    try:
        #print(int(match_i.text))
        match_i.click()
    except:
        continue
    
    #table data
    ls = driver.find_elements_by_class_name('k-grid-table')
    
    
    split_text = ls[0].text.split('\n')
    
    index_headers = split_text[2:6]
    
    #grab first item
    data['MAT_FAMILY'].append(split_text[0])
    data['MATERIAL'].append(split_text[1])
    data['Global Warming'].append(split_text[7])
    data['Eutrophication'].append(split_text[8])
    data['Water Scarcity'].append(split_text[9])
    data['Resource Depletion, Fossil Fuels'].append(split_text[10])
    
    #cycle through remaining items
    for i in range(12, len(split_text), 7):
        material_family = split_text[i]
        material = split_text[i+1]
        ratings = split_text[i+2:i+7]
        
        
        data['MAT_FAMILY'].append(material_family)
        data['MATERIAL'].append(material)
        data['Global Warming'].append(ratings[0])
        data['Eutrophication'].append(ratings[1])
        data['Water Scarcity'].append(ratings[2])
        data['Resource Depletion, Fossil Fuels'].append(ratings[3])



1
2
3
4
5
6
7
8
9
10


In [125]:
#save dataframe
import pandas as pd
pd.DataFrame(data).to_json('../data/HIGG_MSI_data')